<h1 align="center">Python: to async and beyond</h1>
<h4 align="center"><em>Gabor Borics-Kurti</em></h4>

<h2 align="center">A brief introduction to Python</h2>

Python is a high-level language widely used in data science, web development and other areas due to its extensibility and ease of use. This year, it ranked as the 3rd most popular language on the TIOBE index.*[1]*
In the late 1980s, Guido van Rossum started working on an easy-to-use scripting language which he named after the Monthy Python's Flying Circus BBC TV show.

Python 2 was released in 2000 and 3 in 2008 - many applications are still using 2 even though the new versions offer exciting new features (and 2.7 is the last version with support ending in two years). The major changes include:
* Making the print statement a built-in function
* Consolidating duplicate (or very similar) features into unified ones: input and raw_input became input() for example
* Adding support for type hints (more on these later)
* Improving Unicode support by making all strings Unicode by default
* Changing the way integer division is handled: in Python 2, *7 / 2 = 3* while in Python 3 the same expression evaluates to 3.5 - divisions are floating-point ones by default

*[1] https://www.tiobe.com/tiobe-index/*


One of the reasons Python became so popular is its community support and the amount (and quality) of packages available for it. The official, central Python module repository, the Python Package Index (PyPI) contains over 160 000 projects as of December 2018. These cover everything a developer might need from ML toolkits (such as Tensorflow or scikit-learn) through web frameworks (Django, Flask) to game engines (Pygame).

Packages can be installed using pip (the recommended way) or through tools such as the popular Anaconda distribution. As a computer might be used for developing multiple Python applications which use different package and/or Python version, the use of virtual environments is recommended. Virtualenv and the newer (better) Pipenv tools can help with this.

For this workshop I am using the latest Python version (3.7) to be able to show some of the new features it offers.
These include:
* Context variables - safer, more performant storage which can be shared between threads/async processes
* Data classes
* Built-in support for breakpoints
* Nanosecond-resolution time
* Improvements to async features
* Fixing annotations by enabling them to be resolved at runtime

Data classes make it cleaner and easier to declare data models:

In [1]:
class Book:
    title: str
    author: str
    isbn: str
        
    def __init__(self, title, author, isbn = ""):
        self.title = title
        self.author = author
        self.isbn = isbn
        
    def __eq__(self, other):
        return self.title == other.title and self.author == other.author and self.isbn == other.isbn
    
    def __repr__(self):
        return "{}: {}, ISBN {}".format(self.author, self.title, self.isbn)
    
first_book = Book("Think Python: How to Think Like a Computer Scientist", "Allen B. Downey")
second_book = Book("Learn Python 3 the Hard Way", "Zed A. Shaw", "0134692888")
third_book = Book("Think Python: How to Think Like a Computer Scientist", "Allen B. Downey")

print(first_book)
print(second_book)
print(first_book == second_book)
print(first_book == third_book)

Allen B. Downey: Think Python: How to Think Like a Computer Scientist, ISBN 
Zed A. Shaw: Learn Python 3 the Hard Way, ISBN 0134692888
False
True


Comparing this to a dataclass:

In [2]:
from dataclasses import dataclass


@dataclass
class Book:
    title: str
    author: str
    isbn: str = ""
          
first_book = Book("Think Python: How to Think Like a Computer Scientist", "Allen B. Downey")
second_book = Book("Learn Python 3 the Hard Way", "Zed A. Shaw", "0134692888")
third_book = Book("Think Python: How to Think Like a Computer Scientist", "Allen B. Downey")

print(first_book)
print(second_book)
print(first_book == second_book)
print(first_book == third_book)

Book(title='Think Python: How to Think Like a Computer Scientist', author='Allen B. Downey', isbn='')
Book(title='Learn Python 3 the Hard Way', author='Zed A. Shaw', isbn='0134692888')
False
True


<h2 align="center">Lists, generators and performance</h2>

Lists are flexible but can be a bit too slow for certain uses. For example:

In [3]:
def append_grades():
    """ Prints a list of 10 000 grades """
    grades = list(range(10_000)) # 10 000 grades, created using range()

    # Create a new list of grades, adding 1 to each
    new_grades = []
    for grade in grades:
        new_grades.append(grade + 1)

    # Print 3 slices of new and old grades, to verify this operation
    print(grades[:5], grades[5000:5005], grades[-5:])
    print()
    print(new_grades[:5], new_grades[5000:5005], new_grades[-5:])
    

append_grades()

[0, 1, 2, 3, 4] [5000, 5001, 5002, 5003, 5004] [9995, 9996, 9997, 9998, 9999]

[1, 2, 3, 4, 5] [5001, 5002, 5003, 5004, 5005] [9996, 9997, 9998, 9999, 10000]


#### Why is this operation slow?

We have to keep extending the list and call the append function for each iteration - we can make this operation much faster.

In [4]:
import dis


dis.dis(append_grades)

  3           0 LOAD_GLOBAL              0 (list)
              2 LOAD_GLOBAL              1 (range)
              4 LOAD_CONST               1 (10000)
              6 CALL_FUNCTION            1
              8 CALL_FUNCTION            1
             10 STORE_FAST               0 (grades)

  6          12 BUILD_LIST               0
             14 STORE_FAST               1 (new_grades)

  7          16 SETUP_LOOP              26 (to 44)
             18 LOAD_FAST                0 (grades)
             20 GET_ITER
        >>   22 FOR_ITER                18 (to 42)
             24 STORE_FAST               2 (grade)

  8          26 LOAD_FAST                1 (new_grades)
             28 LOAD_METHOD              2 (append)
             30 LOAD_FAST                2 (grade)
             32 LOAD_CONST               2 (1)
             34 BINARY_ADD
             36 CALL_METHOD              1
             38 POP_TOP
             40 JUMP_ABSOLUTE           22
        >>   42 POP_BLOCK

 11     

The same operation is faster using list comprehension because its creation can be optimised and does not have to call the append() function for each Python object we add to it.

In [5]:
def append_grades_faster():
    grades = list(range(10_000)) # 10 000 grades, created using range()

    # Create a new list of grades, adding 1 to each
    new_grades = [grade + 1 for grade in grades]

    # Print 3 slices of new and old grades, to verify this operation
    print(grades[:5], grades[5000:5005], grades[-5:])
    print()
    print(new_grades[:5], new_grades[5000:5005], new_grades[-5:])
    
    
append_grades_faster()

[0, 1, 2, 3, 4] [5000, 5001, 5002, 5003, 5004] [9995, 9996, 9997, 9998, 9999]

[1, 2, 3, 4, 5] [5001, 5002, 5003, 5004, 5005] [9996, 9997, 9998, 9999, 10000]


In [6]:
dis.dis(append_grades_faster)

  2           0 LOAD_GLOBAL              0 (list)
              2 LOAD_GLOBAL              1 (range)
              4 LOAD_CONST               1 (10000)
              6 CALL_FUNCTION            1
              8 CALL_FUNCTION            1
             10 STORE_FAST               0 (grades)

  5          12 LOAD_CONST               2 (<code object <listcomp> at 0x10a1c4810, file "<ipython-input-5-3c1dae8d96f2>", line 5>)
             14 LOAD_CONST               3 ('append_grades_faster.<locals>.<listcomp>')
             16 MAKE_FUNCTION            0
             18 LOAD_FAST                0 (grades)
             20 GET_ITER
             22 CALL_FUNCTION            1
             24 STORE_FAST               1 (new_grades)

  8          26 LOAD_GLOBAL              2 (print)
             28 LOAD_FAST                0 (grades)
             30 LOAD_CONST               0 (None)
             32 LOAD_CONST               4 (5)
             34 BUILD_SLICE              2
             36 BINARY_SUB

We can perform similar operations on dictionaries:


In [7]:
results = {name: grade for name, grade in [("A", 1), ("B", 3)]}
print(results)

{'A': 1, 'B': 3}


And nested lists, or even use conditions to filter data

In [8]:
# Nested list comprehension
all_items = [item for items in [[1,2], [3,4], [5,6]] for item in items]
print(all_items)

# Filtering data
filtered_items = [item for item in all_items if item % 2 == 0]
print(filtered_items)

[1, 2, 3, 4, 5, 6]
[2, 4, 6]


Generators are efficient and can often be used interchangeably with lists:

In [9]:
def generate_list(length):
    for i in range(length):
        yield i
        
        
def generate_long_list(length):
    return (i for i in range(length*100_000))
        

short_list = generate_list(3)
long_list = generate_long_list(3)

# Print short_list
print(short_list)

# Generator objects do not have a length attribute
print(len(long_list))

<generator object generate_list at 0x10a1ea6d8>


TypeError: object of type 'generator' has no len()

In [10]:
print(next(short_list))
print(next(short_list))
print(next(short_list))
print(next(short_list))

0
1
2


StopIteration: 

In [11]:
# We can use next to get the first element of a list too - we can combine this with inline if statements
elements = range(1,100)

# Get the first element divisible by 5
print(next(e for e in elements if e % 5 == 0))

# What happens if we cannot find an element divisible by 200?
try:
    print(next(e for e in elements if e % 200 == 0))
except StopIteration:
    print("StopIteration raised, element not found!")

# Provide a default value so this does not happen
print(next((e for e in elements if e % 200 == 0), "Not found"))

5
StopIteration raised, element not found!
Not found


We can efficiently merge lists and update dictionaries using built-in functions:

In [12]:
first_list = list(range(10))
second_list = ['a', 'b', 'c']

first_list.extend(second_list)
print(first_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 'a', 'b', 'c']


In [13]:
item = {'id': 1, 'weight': 12.5, 'height': 23}
order_details = {'quantity': 4, 'postcode': 'G12 8QQ'}

order_details.update(item)
print(order_details)

{'quantity': 4, 'postcode': 'G12 8QQ', 'id': 1, 'weight': 12.5, 'height': 23}


Or merge/separate a list of tuples:

In [14]:
first_list = range(10)
second_list = range(10, 20)

# Zip returns an iterator
zipped = list(zip(first_list, second_list))

print(zipped)

# Split a list of tuples into separate lists
a, b = zip(*zipped)
print(a)
print(b)

[(0, 10), (1, 11), (2, 12), (3, 13), (4, 14), (5, 15), (6, 16), (7, 17), (8, 18), (9, 19)]
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)
(10, 11, 12, 13, 14, 15, 16, 17, 18, 19)


Iteration using enumerate and itertools:

In [15]:
grades = range(10, 0, -1)

for index, grade in enumerate(grades):
    print("Index: {} grade: {}".format(index, grade))

Index: 0 grade: 10
Index: 1 grade: 9
Index: 2 grade: 8
Index: 3 grade: 7
Index: 4 grade: 6
Index: 5 grade: 5
Index: 6 grade: 4
Index: 7 grade: 3
Index: 8 grade: 2
Index: 9 grade: 1


In [16]:
from itertools import cycle, islice, permutations

# Cycle over the string 'workshop'
workshop = cycle("workshop")

count = 0
while count < 20:
    print(next(workshop), end = " ")
    count += 1

w o r k s h o p w o r k s h o p w o r k 

In [17]:
# Efficiently slice a list
items = [1, 2, 'A', 5, 7, 1, 'C']
sliced_items = islice(items, 1, None, 2)

print(list(sliced_items))

[2, 5, 1]


In [18]:
# Get all permutations of the list of items above
item_perms = permutations(items, 2)

print(list(item_perms))

[(1, 2), (1, 'A'), (1, 5), (1, 7), (1, 1), (1, 'C'), (2, 1), (2, 'A'), (2, 5), (2, 7), (2, 1), (2, 'C'), ('A', 1), ('A', 2), ('A', 5), ('A', 7), ('A', 1), ('A', 'C'), (5, 1), (5, 2), (5, 'A'), (5, 7), (5, 1), (5, 'C'), (7, 1), (7, 2), (7, 'A'), (7, 5), (7, 1), (7, 'C'), (1, 1), (1, 2), (1, 'A'), (1, 5), (1, 7), (1, 'C'), ('C', 1), ('C', 2), ('C', 'A'), ('C', 5), ('C', 7), ('C', 1)]


 <h2 align="center">Data and strings</h2>
 
<h3 align="center">Data</h3>

 We can easily swap two variables, without the need for a temporary variable:


In [19]:
a = 5
b = 6

b, a = a, b
print(a, b)

6 5


For this operation, we have used tuples, the immutable Python array

In [20]:
t = (1,2,3)
print(t)

t[1] = 100

(1, 2, 3)


TypeError: 'tuple' object does not support item assignment

We can use tuples to store data, naming each position:

In [21]:
from collections import namedtuple

person = namedtuple('Person', ['age', 'height', 'weight'])

p = person(99, height=170, weight=70)
print(p.age)

99


Another useful built-in type is the set (and immutable frozenset):

In [22]:
items = range(10)
set_of_items = set(items)

print(len(items) == len(set_of_items))

grades = [1,1,2,3,4,4,4,5]
set_of_grades = frozenset(grades)

print(set_of_grades)

# Common set operations are supported and are efficient:
print(set_of_items.intersection(set_of_grades))
print()
print(set_of_items.difference(set_of_grades))

True
frozenset({1, 2, 3, 4, 5})
{1, 2, 3, 4, 5}

{0, 6, 7, 8, 9}


The beauty of Python comes from its simplicity and elegance - we can use lists as stacks without much work:

In [23]:
stack = ['a', 'b', 'c']

# Push to the stack
stack.append('d')
print(stack)

# Pop from the stack
item = stack.pop()
print(item)
print(stack)

['a', 'b', 'c', 'd']
d
['a', 'b', 'c']


For more efficient data structures (in data science projects for example), there are libraries such as NumPy or pandas

<h3 align="center">Strings</h3>

We can efficiently join lists of characters:

In [24]:
characters = [c for c in "abcdefghijklmnopqrstuvwxyz"]

print(";".join(characters))

a;b;c;d;e;f;g;h;i;j;k;l;m;n;o;p;q;r;s;t;u;v;w;x;y;z


There are many ways to format strings in Python:


In [25]:
# The beginner's way, not efficient
name = "Python 3"
age = 10

print("This is " + name + ", who is " + str(age) + " years old.")

This is Python 3, who is 10 years old.


In [26]:
# Slightly better
print("This is %s, who is %d years old." % (name, age))

This is Python 3, who is 10 years old.


In [27]:
# The preferred way of formatting strings until f-strings were added in Python 3.6
print("This is {name}, who is {age} years old.".format(age = age, name = name))

This is Python 3, who is 10 years old.


In [28]:
# The best option: f-strings
print(f"This is {name}, who is {age} years old")

This is Python 3, who is 10 years old


<h2 align="center">'Type safety'</h2>

As Python is dynamically typed, we can run into typing-related bugs:

In [31]:
import random


def create_a_or_b():
    """ Simulates a type-related bug """
    i = random.randint(0, 10)
    return A() if i >= 5 else B()

class A:
    def print_a(self):
        print("A")
        
class B:
    def print_b(self):
        print("B")
        
# We assume that variable a has type A but cannot be certain if it is coming from somewhere else...
a = create_a_or_b()

a.print_a()

AttributeError: 'B' object has no attribute 'print_a'

Since Python 3.5, developers can add type annotations to their code to aid static type checkers in catching mistakes. Adding such type hints will not make Python typed, but helps avoid common issues.

In [32]:
from typing import Dict

# Maps users to some external ID
custom_user_map = Dict[str, int]


def get_external_id(username: str, usermap: custom_user_map) -> int:
    """ Returns the external ID of the given user """
    return usermap.get(username)


print(get_external_id('test', {'test1': 1}))
print(get_external_id(1, {'test': 1}))

None
None


<h2 align="center">Kwargs and lambdas</h2>

When looking through Python library code, we can come across \*args and \**kwargs, but how do we use them?

In [33]:
def call_fn_by_name_with_arg_list(name, args):
    """ Calls the given function name, passing args to it """
    globals()[name](*args, should_print = True)
    

def print_args(*args, should_print):
    """ Prints args if should_print is True, does nothing otherwise """
    if should_print:
        print(f"Args: {args}")
        
        
call_fn_by_name_with_arg_list("print_args", ['this', 'is', 'a', 'list', 'of', 'args'])

Args: ('this', 'is', 'a', 'list', 'of', 'args')


In [34]:
def call_fn_by_name_with_kwargs(name, kwargs):
    """ Calls the given function name, passing kwargs to it """
    globals()[name](**kwargs, should_print = True)
    
    
def print_kwargs(should_print, **kwargs):
    """ Prints kwargs if should_rpint is True, does nothing otherwise """
    if should_print:
        print(f"Name: {kwargs.get('name')}, age: {kwargs.get('age')}")
    
    
call_fn_by_name_with_kwargs("print_kwargs", {'name': 'Python 3', 'age': 10})

Name: Python 3, age: 10


There are plenty of built-in functions to aid functional programming in Python, I show a very limited set of them below.

Lambdas are probably the best-known:

In [35]:
items = [{'a': 2}, {'a': 9}, {'a': 1}, {'a': 5}]

print(sorted(items, key = lambda k: k['a']))

# This is not Pythonic, the second version is preferred
filter_even_numbers = lambda num: num % 2 == 0
print([num for num in range(10) if filter_even_numbers(num)])

# Preferred option
def filter_even_numbers(num): return num % 2 == 0
print([num for num in range(10) if filter_even_numbers(num)])

[{'a': 1}, {'a': 2}, {'a': 5}, {'a': 9}]
[0, 2, 4, 6, 8]
[0, 2, 4, 6, 8]


Filter an iterable:

In [36]:
filtered_items = filter(lambda item: item['a'] % 3 == 0, items)
print(list(filtered_items))

[{'a': 9}]


Check if all or any of the given conditions hold:

In [37]:
print(all((1 < 3, 4 < 5, 6 > 3)))

True


In [38]:
print(any((2 > 3, 5 < 6)))

True


Apply the same function to all elements

In [39]:
ints = range(10)
strings = list(map(str, ints))

# Strings are really strings
for s in strings:
    if not isinstance(s, str):
        print(f"{s} is not a string!")
        
print(strings)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


Reduce an iterable to a value:

In [40]:
from functools import reduce

# Sum of squares - 338350
first_100_numbers = range(1, 101)
sum_of_squares = reduce(lambda sum, elem: sum + elem ** 2, first_100_numbers)

print(sum_of_squares)

338350


<h2 align="center">OOP in Python</h2>

While Python is not a typically object-oriented language like Java, it does support object-oriented programming in the Pythonic way.

There is no need for interfaces due to its support for multiple inheritance, abstract classes can achieve the same results:

In [41]:
from abc import ABC, abstractmethod


class AbstractItem(ABC):
    @abstractmethod
    def price(self):
        raise NotImplementedError("Price() not defined in AbstractItem")
      
    
class Item(AbstractItem):   
    def price(self):
        return 42
    

a = Item()
a.price()

42

Defining classes is simple, and enums are also supported

In [42]:
from enum import Enum


class Colours(Enum):
    RED = "red"
    GREEN = "green"
    BLUE = "blue"


class Car:
    wheels: int = 4
    colour: Colours
        
    def __init__(self, colour):
        self.colour = colour
        
    def drive(self):
        return f"Driving a {self.colour.value} car with {self.wheels} wheels"
    
    
c = Car(Colours.RED)
print(c.drive())

Driving a red car with 4 wheels


Object equality behaves the 'usual' way:

In [43]:
# Reusing the car class from above
red_car = Car(Colours.RED)
another_red_car = Car(Colours.RED)

print(red_car is another_red_car)
print(red_car is red_car)

False
True


In [44]:
print(red_car == another_red_car)

False


In [45]:
class EqualCar(Car):   
    def __eq__(self, other):
        return all((self.wheels == other.wheels, self.colour == other.colour))
    

red_car = EqualCar(Colours.RED)
another_red_car = EqualCar(Colours.RED)

print(red_car == another_red_car)

True


It is important to note that **everything in Python is an object**.

<h2 align="center">To async and beyond</h2>

The Global Interpreter Lock (GIL) ensures that every CPython object is thread-safe which effectively means that every CPython interpreter instance is limited to using one thread. In some cases, this is not a huge limitation, for example a lightweight web application might run as several CPython instances. In more CPU-heavy applications using multiple threads without the need to work around the GIL would be a good way to improve usability and performance. Until the GIL is finally retired, there is an easy-to-use, performant solution for I/O bound applications: async and await.


There are different ways to be asynchronous in Python:
* Callbacks: just like in the Tornado library, fetching a resource with Tornado happens asynchronously and calls a function passed to it when done - a callback.
* Green threads: manage threads in software
* Use generators (asyncio and aiohttp): run generators in an event loop using *yield from*
* Use async/await for clarity and better control

A simple asyncio example:

In [46]:
import asyncio


async def api_call(arg):
    """ Sleep then returns arg - simulates an API call """
    await asyncio.sleep(0.1)
    return f"API called with {arg}"


args = range(10)
for arg in args:
    result = await api_call(arg)
    print(result)
        

# In a standard environment we would need to run the main asyncio loop
# asyncio.run(main())

API called with 0
API called with 1
API called with 2
API called with 3
API called with 4
API called with 5
API called with 6
API called with 7
API called with 8
API called with 9


A slightly more async example

In [47]:
async def api_call(arg):
    """ Sleep then returns arg - simulates an API call """
    await asyncio.sleep(1)
    return f"API called with {arg}"


args = range(10)
tasks = [asyncio.create_task(api_call(arg)) for arg in args]

for i in args:
    result = await tasks[i]
    print(result)

API called with 0
API called with 1
API called with 2
API called with 3
API called with 4
API called with 5
API called with 6
API called with 7
API called with 8
API called with 9


We can use timeouts to control the longest time we are willing to wait for a task to complete:

In [48]:
async def api_call(arg):
    """ Sleep then returns arg - simulates an API call """
    await asyncio.sleep(1)
    return f"API called with {arg}"


args = range(10)
tasks = [asyncio.wait_for(api_call(arg), timeout = 0.5) for arg in args]

for i in args:
    result = await tasks[i]
    print(result)

TimeoutError: 

Many other useful strcutures are available in the asyncio library such as locks, queues, priority queues, streams...

Finally, an asynchronous HTTP example using [AIOHTTP](https://aiohttp.readthedocs.io/en/stable/):

In [50]:
from aiohttp import web
import requests


async def handle(request):
    return web.Response(text = "Request handled")

try:
    app = web.Application()
    app.add_routes([web.get('/', handle)])

    # Another workaround for Jupyter notebooks
    runner = web.AppRunner(app)
    await runner.setup()
    site = web.TCPSite(runner, 'localhost', 3002)
    await site.start()
except (OSError, RuntimeError) as e:
    print(f"Error thrown: {e}")
    # Stop the server
    await runner.cleanup()

Error thrown: [Errno 48] error while attempting to bind on address ('127.0.0.1', 3002): address already in use


In [51]:
# Stop the server
await runner.cleanup()

<h2 align="center">Is Python slow?</h2>

A common argument against Python is its speed, and even a well-optimised computation-intensive Python program will definitely be slower than the same well-written program written in C. While Python might not be the language to build operating systems in, it can offer a huge increase in productivity compared to 'faster' languages.

In data-related programs, libraries such as numpy can help by providing efficient data structures and operations on them.

The next steps in optimising Python programs can be the use of alternative implementations such as [PyPy](http://pypy.org/), use [Cython](https://cython.org/) and static typing or a JIT compiler like [Numba](http://numba.pydata.org/). As the standard CPython distribution is based on C, it is also possible to write extension (even entire projects) in C and use Python as a wrapper around them.

Interestingly, there are even Python to JavaScript compilers, for those who would like to write their ReactJS applications in Python: [Transcrypt](http://www.transcrypt.org/) with [PyReact](https://github.com/doconix/pyreact).

<h2 align="center">The future of Python</h2>

Python 3.8 is planned to include the usual performance and security improvements, but also a few new features. 
* Assignment expressions aim to improve style, readability, performance and the debugging experience: [PEP 572](https://www.python.org/dev/peps/pep-0572/)
* Improving the startup speed of the CPython interpreter
* Enabling the use of context variables with generators

After the end of 2019 when these features are released in 3.8, we can expect 3.9 then 4.0 (3.10?) in the next years which are planned to further improve performance (possibly by implementing a JIT compiler like PyPy does) or even getting rid of the GIL. In any case, Python is an extremely popular language and its use in industry is expected to continue growing.

<h2 align="center">Interesting Python resources</h2>

If you would like to learn more about any of these topics or Python in general, these resources might be helpful:
* The excellent Python documentation: https://docs.python.org/
* Python Data Science Handbook: https://jakevdp.github.io/PythonDataScienceHandbook/
* All Python Enhancement Proposals, including proposed ones: https://www.python.org/dev/peps/
* Interesting thoughts on the 'slowness' of Python: https://hackernoon.com/yes-python-is-slow-and-i-dont-care-13763980b5a1

<h2 align="center">Questions?</h2>

<h2 align="center">Thank you!</h2>